**Please upvote my notebook if You like it!**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Downloading data**
+ adding data from "stroke prediction dataset"

In [ ]:
train_df = pd.read_csv('/kaggle/input/playground-series-s3e2/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s3e2/test.csv')
submission = pd.read_csv('/kaggle/input/playground-series-s3e2/sample_submission.csv')
addition_data = pd.read_csv('/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')

In [ ]:
train_df

In [ ]:
addition_data = addition_data[addition_data['stroke']==1]
addition_data

There is missing data in addition dataset

In [ ]:
addition_data.isna().any()

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

def knn_impute(df, na_target):
    df = df.copy()
    
    numeric_df = df.select_dtypes(np.number)
    non_na_columns = numeric_df.loc[: ,numeric_df.isna().sum() == 0].columns
    
    y_train = numeric_df.loc[numeric_df[na_target].isna() == False, na_target]
    X_train = numeric_df.loc[numeric_df[na_target].isna() == False, non_na_columns]
    X_test = numeric_df.loc[numeric_df[na_target].isna() == True, non_na_columns]
    
    knn = KNeighborsRegressor()
    knn.fit(X_train, y_train)
    
    y_pred = knn.predict(X_test)
    
    df.loc[df[na_target].isna() == True, na_target] = y_pred
    
    return df

na_cols = [col for col in addition_data.columns if addition_data[col].isnull().sum()!=0]

for col in na_cols:  
    addition_data = knn_impute(addition_data, col)

In [ ]:
addition_data.isna().any()

In [ ]:
# addition_data.bmi = addition_data.bmi.fillna(np.mean(addition_data.bmi))

In [ ]:
# train_df['generated'] = 1
# test_df['generated'] = 1
# addition_data['generated'] = 0
train_df = pd.concat([train_df, addition_data],axis=0, ignore_index=True)

In [ ]:
train_df

In [ ]:
train_df['stroke'].hist()

No missing data in our dataset

In [ ]:
train_df.isna().any()

In [ ]:
train_df.info()

In [ ]:
df = pd.concat([train_df, test_df], axis=0)
df

In [ ]:
# df['morbid'] = np.where(df.bmi>40,1,0)
# df['obese'] = np.where(df.bmi>30,1,0)

In [ ]:
# def feature_risk_factors(df):
#     df["risk_factors"] = df[[
#         "avg_glucose_level", "age", "bmi", 
#         "hypertension", "heart_disease", 
#         "smoking_status"
#     ]].apply(
#         lambda x: \
#         0 + (1 if x.avg_glucose_level > 99 else 0) + \
#         (1 if x.age > 45 else 0) + (1 if x.bmi > 24.99 else 0) + \
#         (1 if x.hypertension == 1 else 0) + \
#         (1 if x.heart_disease == 1 else 0) + \
#         (1 if x.smoking_status in ["formerly smoked", "smokes"] else 0),
#         axis=1
#     )
#     return df

# feature_risk_factors(df)

In [ ]:
df = pd.get_dummies(df)
df = df.drop('id', axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

num_cols = ['age', 'avg_glucose_level', 'bmi']

y = df['stroke']
df = df.drop('stroke', axis=1)

df[df.columns] = scaler.fit_transform(df[df.columns])

# df = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)

In [ ]:
df

In [ ]:
train_df = df.iloc[:-len(test_df),:]
train_df['stroke'] = y[:-len(test_df)]
test_df = df.iloc[-len(test_df):,:].reset_index(drop=True)

X = train_df.drop('stroke', axis=1)
y = train_df.stroke
X_test = test_df

In [ ]:
# y = y * 2

In [ ]:
test_df

I am using my ensemble model from previous competition

**CatBoost model**

In [ ]:
import catboost
from sklearn.model_selection import KFold, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score

n_folds = 11 #12

MAX_ITER = 15000
PATIENCE = 1000
DISPLAY_FREQ = 100

eval_predsCB = []
predsCB = []

# k_fold = StratifiedKFold(n_splits=n_folds, random_state=42, shuffle=True)
k_fold = RepeatedStratifiedKFold(n_splits=n_folds, n_repeats=20, random_state=42) # 20

MODEL_PARAMS = {
                'random_seed': 1234,             
                'iterations': MAX_ITER,
                'early_stopping_rounds': PATIENCE,
                'use_best_model': True,
                'eval_metric': 'AUC',
#                 'eval_metric': 'RMSE',
                'verbose': 1000,
    
                 'depth': 3,
                 'learning_rate': 0.01,
                 'rsm': 0.5,
                 'subsample': 0.931,
                 'l2_leaf_reg': 69,
                 'min_data_in_leaf': 20,
                 'random_strength': 0.175,
               }


for train_index, test_index in k_fold.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
#     model = catboost.CatBoostClassifier(**MODEL_PARAMS)
    model = catboost.CatBoostRegressor(**MODEL_PARAMS)
    
    model.fit(X=X_train, y=y_train,
          eval_set=[(X_valid, y_valid)],
          early_stopping_rounds = PATIENCE,
#           metric_period = DISPLAY_FREQ
         )
    predsCB.append(model.predict(X_test))
    #     eval_predsCB.append(model.predict(X))


**XGBoost model**

In [ ]:
from xgboost import XGBClassifier, XGBRegressor

# n_folds = 20
k_fold = RepeatedStratifiedKFold(n_splits=n_folds, n_repeats=20, random_state=42) # 20

eval_predsXB = []
predsXB = []

PATIENCE = 200

MODEL_PARAMS = {       'n_estimators': 1000, #1000, 5000
#                        'learning_rate': 0.05,
                       'max_depth': 4, # 3
                       'colsample_bytree': 0.9, # 0.95
                       'subsample': 1,
                       'reg_lambda': 20,
                       'eval_metric': 'auc',
#                        'eval_metric': 'rmse',
                       'early_stopping_rounds': PATIENCE,
#                        'tree_method': 'gpu_hist',
                       'seed': 1
}

for train_index, test_index in k_fold.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
#     model = XGBClassifier(**MODEL_PARAMS)
    model = XGBRegressor(**MODEL_PARAMS)
    
    model.fit(X=X_train, y=y_train,
          eval_set=[(X_valid, y_valid)],
#           early_stopping_rounds = PATIENCE,
          verbose = 100
         )
    predsXB.append(model.predict(X_test))
#     eval_predsXB.append(model.predict(X))


**LGBM model**

In [ ]:
import lightgbm as lgbm
from lightgbm.sklearn import LGBMClassifier, LGBMRegressor

# n_folds = 20
k_fold = RepeatedStratifiedKFold(n_splits=n_folds, n_repeats=20, random_state=42) # 20

eval_predsLB = []
predsLB = []

MODEL_PARAMS = {
                       'learning_rate': 0.01,
                       'max_depth': 9,
                       'num_leaves': 90,
                       'colsample_bytree': 0.8,
                       'subsample': 0.9,
                       'subsample_freq': 5,
                       'min_child_samples': 36,
                       'reg_lambda': 28,
                       'n_estimators': 20000,
                       'metric': 'auc',
#                        'metric': 'rmse',
                       'random_state': 1
}

callbacks = [lgbm.early_stopping(30, verbose=1), lgbm.log_evaluation(period=0)]

for train_index, test_index in k_fold.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
#     model = lgbm.LGBMClassifier(**MODEL_PARAMS)
    model = lgbm.LGBMRegressor(**MODEL_PARAMS)
    
    model.fit(X=X_train, y=y_train,
          eval_set=[(X_valid, y_valid)],
#           early_stopping_rounds = PATIENCE,
          callbacks=callbacks
         )
    predsLB.append(model.predict(X_test))
#     eval_predsLB.append(model.predict(X))

**Lasso regression**

In [ ]:
from sklearn.linear_model import LassoCV

# n_folds = 20
k_fold = RepeatedStratifiedKFold(n_splits=n_folds, n_repeats=20, random_state=42) # 20

eval_predsLR = []
predsLR = []

MODEL_PARAMS = {
                       'precompute': "auto",
                       'fit_intercept': True,
                       'normalize': False,
                       'max_iter': 1000,
                       'verbose': False,
                       'eps': 1e-04,
                       'cv': 5,
                       'n_alphas': 1000,
                       'n_jobs': 8,
}

callbacks = [lgbm.early_stopping(30, verbose=1), lgbm.log_evaluation(period=0)]

for train_index, test_index in k_fold.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
#     model = lgbm.LGBMClassifier(**MODEL_PARAMS)
    model = LassoCV(**MODEL_PARAMS)
    
    model.fit(X=X_train, y=y_train,
#           eval_set=[(X_valid, y_valid)],
#           early_stopping_rounds = PATIENCE,
#           callbacks=callbacks
         )
    predsLR.append(model.predict(X_test))
#     eval_predsLB.append(model.predict(X))

**Keras NN**
from amazing notebook https://www.kaggle.com/code/kirillka95/ps-s3e02-keras-nn-kfold-pub

In [ ]:
from tensorflow import keras
import tensorflow_addons as tfa
from tensorflow.keras import layers

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input, Lambda, Concatenate, Add, BatchNormalization, LeakyReLU

def my_model():
    x_input = Input(shape=(X.shape[-1]), name='input')
    x1 = Dense(256, activation='relu')(x_input)
    d1 = Dropout(0.1)(x1)
    x2 = Dense(128, activation='relu')(d1)
    d2 = Dropout(0.1)(x2)
    x3 = Dense(64, activation='relu')(d2)
    d3 = Dropout(0.1)(x3)
    output = Dense(1, activation='sigmoid', name='output')(d3)
    
    model = Model(x_input, output, name='nn_model')
    return model


VERBOSE = False
BATCH_SIZE = 64
predictions, scores = [], []

lr = keras.callbacks.ReduceLROnPlateau(monitor="val_auc", 
                                       mode='max', 
                                       patience=3, 
                                       factor=0.1, 
                                       min_lr=1e-6, 
                                       min_delta=0.0001)

es = keras.callbacks.EarlyStopping(monitor="val_auc", 
                                   mode='max', 
                                   patience=7, 
                                   min_delta=0.0001, 
                                   restore_best_weights = True)

# k_fold = StratifiedKFold(n_splits=n_folds, random_state=42, shuffle=True)
k_fold = RepeatedStratifiedKFold(n_splits=n_folds, n_repeats=5, random_state=42) # 5

eval_predsNN = []
predsNN = []


for train_index, test_index in k_fold.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
    model = my_model()
    model.compile(optimizer='adam', 
                  loss=tfa.losses.SigmoidFocalCrossEntropy(alpha=0.8, gamma=2), # alpha=0.8, gamma=2
                  metrics='AUC')
        
    model.fit(X_train, y_train, 
              validation_data=(X_valid, y_valid), 
              epochs=500,
              verbose=VERBOSE,
              batch_size=BATCH_SIZE, 
              class_weight={0: 1, 1: 10},
              callbacks=[lr, es])
    
    predsNN.append(model.predict(X_test))

LassoCV preds from amazinf notebook https://www.kaggle.com/code/tilii7/modeling-stroke-dataset-with-lasso-regression

In [ ]:
lasso_preds = pd.read_csv('/kaggle/input/lassocv-submission/submission (2).csv')
predLS = lasso_preds['stroke']

In [ ]:
def normalizing_predictions(model_predictions):
    return (model_predictions - np.min(model_predictions)) / (np.max(model_predictions) - np.min(model_predictions))

In [ ]:
a = 0.08  #0.15 < 0.05 < 0.08 > 0.09 > 0.1 > 0.11 > 0.125
b = 0.08  #0.1 < 0.05 < 0.08 > 0.09 > 0.1 > 0.11 > 0.125
c = 0.08  #0.15 < 0.05 < 0.08 > 0.09 > 0.1 > 0.11 > 0.125
d = 0.08  #0 < 0.05 < 0.08 > 0.09 > 0.1 > 0.11 > 0.125
e = 0.24    #0.2 < 0.3 < 0.24 > 0.22 > 0.2 > 0.2 > 0.2
f = 0.44    #0.4 < 0.5 < 0.44 > 0.42 > 0.4 > 0.36 > 0.3
predCB = np.average(np.array(predsCB),axis=0).clip(0,1)
predXB = np.average(np.array(predsXB),axis=0).clip(0,1)
predLB = np.average(np.array(predsLB),axis=0).clip(0,1)
predLR = np.average(np.array(predsLR),axis=0).clip(0,1)
predNN = np.average(np.array(predsNN),axis=0).clip(0,1)

# predCB = np.average(np.array(predsCB),axis=0)
# predXB = np.average(np.array(predsXB),axis=0)
# predLB = np.average(np.array(predsLB),axis=0)
# predLR = np.average(np.array(predsLR),axis=0)
# predNN = np.average(np.array(predsNN),axis=0).clip(0,1)

# predCB = normalizing_predictions(predCB)
# predXB = normalizing_predictions(predXB)
# predLB = normalizing_predictions(predLB)
# predLR = normalizing_predictions(predLR)
# predNN = normalizing_predictions(predNN)

predNN = pd.Series(*predNN.T)
predNN = np.array(predNN)
pred = predCB * a + predXB * b + predLB * c + predLR * d + predNN * e + predLS * f

In [ ]:
predLS.hist()

In [ ]:
pd.Series(predCB).hist()

In [ ]:
pd.Series(predXB).hist()

In [ ]:
pd.Series(predLB).hist()

In [ ]:
pd.Series(predLR).hist()

In [ ]:
pd.Series(predNN).hist()

In [ ]:
pred

In [ ]:
pd.Series(pred).hist(bins=20)

**Making submission**

In [ ]:
submission['stroke'] = pred
submission

In [ ]:
submission.to_csv('submission.csv', index=False)